<a href="https://colab.research.google.com/github/Licencinho/PMA/blob/main/run_PMA_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask-wtf

In [12]:
!ngrok authtoken "26BvbMOktqjiQ2Ibfb05r7jzTcL_4UJaagneSjXiB6qeUZXzT"

Authtoken saved to configuration file: C:\Users\alons/.ngrok2/ngrok.yml


In [5]:
#only execute in Google Colab. Not needed in local
!git clone "https://github.com/Licencinho/PMA"
import os
os.chdir("/content/PMA")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '/content/PMA'

In [29]:
from flask import *
from flask_ngrok import run_with_ngrok
import pandas as pd
from flask_wtf import FlaskForm
from wtforms import *
from wtforms import validators
from wtforms.validators import *


app = Flask(__name__)
app.config['SECRET_KEY'] = 'super-secret-impossible-impopossible' # don't do this in production

from flask_wtf.csrf import CSRFProtect
csrf = CSRFProtect(app)

app_data = {
    "name":         "Dissertation Planner",
    "description":  "A basic Flask app for students to plan their dissertations",
    "author":       "Peter Simeth",
    "html_title":   "Dissertation Planner",
    "project_name": "Dissertation Planner",
    "keywords":     "flask, webapp, dissertation, planning, python"
}

class UserForm(FlaskForm):
	module_number = IntegerField('How many modules does your Master have? (Required)', validators=[NumberRange(min=5, max=15, message='Please put a number between 5 and 15')], default=0)
	hours_in_dissertation = IntegerField('How many hours have you worked in your dissertation so far?', default=0)
	module_1_boolean = BooleanField('Module 1')
	module_2_boolean = BooleanField('Module 2')
	module_3_boolean = BooleanField('Module 3')
	module_4_boolean = BooleanField('Module 4')
	module_5_boolean = BooleanField('Module 5')
	module_6_boolean = BooleanField('Module 6')
	module_7_boolean = BooleanField('Module 7')
	module_8_boolean = BooleanField('Module 8')
	module_9_boolean = BooleanField('Module 9')
	module_10_boolean = BooleanField('Module 10')
	module_1_PMA_percentage = IntegerField('Module 1', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_2_PMA_percentage = IntegerField('Module 2', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_3_PMA_percentage = IntegerField('Module 3', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_4_PMA_percentage = IntegerField('Module 4', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_5_PMA_percentage = IntegerField('Module 5', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_6_PMA_percentage = IntegerField('Module 6', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_7_PMA_percentage = IntegerField('Module 7', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_8_PMA_percentage = IntegerField('Module 8', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_9_PMA_percentage = IntegerField('Module 9', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_10_PMA_percentage = IntegerField('Module 10', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)


@app.route('/')
def index():
    return render_template('index.html', app_data=app_data)


@app.route('/calendar')
def calendar():
    df = pd.read_csv("NamesTwo.csv")
    return render_template('calendar.html', df=df, app_data=app_data)


@app.route('/workload')
def workload():
    form = UserForm()
    return render_template('workload.html', form = form, app_data=app_data)


@app.route('/submit', methods=('GET', 'POST'))
def submit():
	form = UserForm()
	if request.method == "POST":
		if form.validate_on_submit():
			print("I'm in form validation")
			# max_id = df['ID'].max()
			# new_id = max_id + 1
			# df2 = pd.DataFrame({'ID':[new_id], 
			#	'Name':[request.form.get("name")],
			#	'Email':[request.form.get("email")],
			#	'Age':[request.form.get("dob")],
			#	'Specialism':[request.form.get("specialism")]
			# })
			# df2['Age'] = pd.to_datetime(df2['Age'], dayfirst=True)
			# now = pd.Timestamp('now')
			# df2['Age'] = (now - df2['Age']).astype('<m8[Y]')
			# df = pd.concat([df, df2], ignore_index=True)
			# df.to_csv("NamesTwo.csv", index=False)

			return render_template('results.html', form=form, app_data=app_data)

	return render_template('workload.html', form=form, app_data=app_data)
	

run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5644-31-205-0-51.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Mar/2022 15:59:47] "GET /workload HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2022 16:00:10] "POST /submit HTTP/1.1" 200 -


I'm in form validation
